In [ ]:
from langchain.output_parsers import StructuredOutputParser, PydanticOutputParser, ResponseSchema
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from pydantic import BaseModel
from typing import List

## 1. Classification code definition explanation

In [ ]:
CPC_interp_response_schemas = [
        ResponseSchema(
            name="Technology Feature",
            type="str",
            description="Describe the technology feature of this Classification Code in one sentence. Retain only essential details."
        ),
        ResponseSchema(
            name="Functions",
            type="List[str]",
            description="Several phrases describe the functions that the Classification Code can achieve, particularly supporting the Design Requirement."
        ),
        ResponseSchema(
            name="Core Feature",
            type="str",
            description="A phrase that describes the core technology feature of this Classification Code."
        )
    ]
CPC_interp_output_parser = StructuredOutputParser.from_response_schemas(CPC_interp_response_schemas)
CPC_interp_format_instructions = CPC_interp_output_parser.get_format_instructions(only_json=True)

CPC_interp_system_prompt = SystemMessagePromptTemplate.from_template(
    "You are an expert in patent analysis, have a detailed understanding of the CPC patent classification system, and can describe the technology feature of a Classification Code in detailed professional language. You must return a valid JSON format response.")
CPC_interp_requirement = """
Task: Comprehensively understand the following Classification Code definition, and professionally and logically describe the technology feature and function of the Classification Code.

**Classification Code:** {cpc}
**Definition:** {cpc_definition}
Note: The symbol "->" indicates a refinement of technology features. For example, if Classification Code 1 is defined as "Technology Feature A -> Technology Feature B", it means that under the condition of Technology Feature A, it is limited to Technology Feature B.

The output should be formatted as a JSON instance that conforms to the JSON schema below.
{CPC_interp_format_instructions}
"""
CPC_interp_human_prompt = HumanMessagePromptTemplate.from_template(CPC_interp_requirement)
CPC_interp_prompt = ChatPromptTemplate.from_messages([CPC_interp_system_prompt, CPC_interp_human_prompt])

## 2. Explanation of innovation opportunities

In [ ]:
P_CSG_response_schemas = [
        ResponseSchema(
            name="Title",
            type="str",
            description="The title of conceptual solution, describing the core functions and purposes of the solution."
        ),
        ResponseSchema(
            name="Integration",
            type="str",
            description="A brief introduction summarizing the technologies integrated into this solution."
        ),
        ResponseSchema(
            name="Core Functions",
            type="List[str]",
            description="Several phrases that describe the core functions of the conceptual solution."
        ),
        ResponseSchema(
            name="Purposes",
            type="str",
            description="A brief description of the purposes of the conceptual solution."
        )
    ]
P_CSG_output_parser = StructuredOutputParser.from_response_schemas(P_CSG_response_schemas)
P_CSG_format_instructions = P_CSG_output_parser.get_format_instructions(only_json=True)

P_CSG_system_prompt = SystemMessagePromptTemplate.from_template(
    "You are a conceptual design expert specializing in identifying innovation opportunities through the analysis of combined patent Classification Codes. You must return a valid JSON format response.")
P_CSG_requirement = """
Task: Analyze the innovation opportunity arising from the technological convergence represented by the two Classification Codes in Technology Convergence Prediction, and develop an innovative conceptual solution that meets the Design Requirement.

## Design Requirement
{requirement}

## Technology Convergence Prediction
Predict that the following two Classification Codes will undergo technology convergence, combining to generate innovation opportunity.

{cpc1}: {cpc1_core_feature}
technology feature: {cpc1_technology_feature}
function: {cpc1_technology_feature}

{cpc2}: {cpc2_core_feature}
technology feature: {cpc2_technology_feature}
function: {cpc2_technology_feature}

Based on the technologies represented by these two Classification Codes, consider the technological innovations that could result from integrating {cpc2_core_feature} into {cpc1_core_feature}.

The output should be formatted as a JSON instance that conforms to the JSON schema below.
{P_CSG_format_instructions}
"""
P_CSG_human_prompt = HumanMessagePromptTemplate.from_template(P_CSG_requirement)
P_CSG_prompt = ChatPromptTemplate.from_messages([P_CSG_system_prompt, P_CSG_human_prompt])

## 3.Technology Function Analysis

In [ ]:
TFA_response_schemas = [
        ResponseSchema(
            name="Core Function",
            type="List[str]",
            description="Several phrases that describe the core functions of the classification code, particularly supporting the conceptual solution. Retain only essential details."
        )
    ]
TFA_output_parser = StructuredOutputParser.from_response_schemas(TFA_response_schemas)
TFA_format_instructions = TFA_output_parser.get_format_instructions(only_json=True)

TFA_system_prompt = SystemMessagePromptTemplate.from_template(
    "You are a technical analysis expert responsible for analyzing the functions of the Classification Code that support the implementation of the Conceptual Solution.  You must return a valid JSON format response.")
TFA_requirement = """
Task: Analyze the core functions of the Classification Code in the Conceptual Solution.

## Design Requirement
{requirement}

## Conceptual Solution
{conceptual_solution}

## Classification Code
{cpc}: {cpc_core_feature}
function: {cpc_function}

The output should be formatted as a JSON instance that conforms to the JSON schema below.
{TFA_format_instructions}
"""
TFA_human_prompt = HumanMessagePromptTemplate.from_template(TFA_requirement)
TFA_prompt = ChatPromptTemplate.from_messages([TFA_system_prompt, TFA_human_prompt])

## 4. Preliminary Concept Design

In [ ]:
CSO_response_schemas = [
        ResponseSchema(
            name="Title",
            type="str",
            description="A concise title that summarizes the Conceptual Solution, providing a clear overview of the solution's core functions and purposes."
        ),
        ResponseSchema(
            name="Function Modules",
            type="List[str]",
            description="Several phrases that describe the core function modules of the conceptual solution, with 2-5 functional modules."
        ),
        ResponseSchema(
            name="Purpose",
            type="str",
            description="A brief description of the purposes and the innovativeness of the conceptual solution."
        ),
    ]
CSO_output_parser = StructuredOutputParser.from_response_schemas(CSO_response_schemas)
CSO_format_instructions = CSO_output_parser.get_format_instructions(only_json=True)

CSO_system_prompt = SystemMessagePromptTemplate.from_template(
    "You are a conceptual design expert who develops innovative conceptual solutions. You must return a valid JSON format response.")
CSO_requirement = """
Task: Consider the support of Relevant Technologies for the Preliminary Conceptual Solution and develop a professional and innovative conceptual solution.

## Design Requirement
{requirement}

## Preliminary Conceptual Solution
{preliminary_conceptual_solution}

## Relevant Technologies
{relevant_technologies}

The output should be formatted as a JSON instance that conforms to the JSON schema below.
{CSO_format_instructions}
"""
CSO_human_prompt = HumanMessagePromptTemplate.from_template(CSO_requirement)
CSO_prompt = ChatPromptTemplate.from_messages([CSO_system_prompt, CSO_human_prompt])

## 5. Function Module Design

In [ ]:
class FunctionalModule(BaseModel):
    module_name: str
    description: str
    applied_technologies: List[str]

class FunctionalModulesResponse(BaseModel):
    functional_modules: List[FunctionalModule]

FMD_output_parser = PydanticOutputParser(pydantic_object=FunctionalModulesResponse)
FMD_format_instructions = FMD_output_parser.get_format_instructions()

FMD_system_prompt = SystemMessagePromptTemplate.from_template(
    "You are a conceptual design expert specializing in designing functional modules required for Conceptual Solutions. You must return a valid JSON format response.")
FMD_requirement = """
Task: Design the functional modules of the Conceptual Solution.

## Design Requirement
{requirement}

## Conceptual Solution
{conceptual_solution}

## Relevant Technologies
{relevant_technologies}

Analyze the mapping relationship between Relevant Technologies and the Function Modules in Conceptual Solution, and design Function Modules that meet the Design Requirement. Maximize the utilization of all technologies listed in Relevant Technologies.
For each module, provide:
- **Module Name**: The name of the functional module.
- **Description**: A detailed description of the functional module, including the working principles, functions, and purposes of the module.
- **Applied Technologies**: The List of Relevant Technologies represented by classification codes that are applied in the functional module.

{FMD_format_instructions}
"""
FMD_human_prompt = HumanMessagePromptTemplate.from_template(FMD_requirement)
FMD_prompt = ChatPromptTemplate.from_messages([FMD_system_prompt, FMD_human_prompt])

## 6. Conceptual Solution Design

In [ ]:
CSD_response_schemas = [
        ResponseSchema(
            name="Title",
            type="str",
            description="A concise title that summarizes the Conceptual Solution, providing a clear overview of the solution's core functions and purposes. Retain only essential details."
        ),
        ResponseSchema(
            name="Description",
            type="str",
            description="A detailed description of the conceptual solution includes an overall functional overview, and the primary functions and purposes of each functional module. Retain only the core details and present the content in a single paragraph."
        )
    ]
    CSD_output_parser = StructuredOutputParser.from_response_schemas(CSD_response_schemas)
    CSD_format_instructions = CSD_output_parser.get_format_instructions(only_json=True)

    CSD_system_prompt = SystemMessagePromptTemplate.from_template(
        "You are a conceptual design expert who optimizes and adjusts conceptual solutions by analyzing the descriptions of functional modules, ensuring logical coherence and professional articulation. You must return a valid JSON format response.")
    CSD_requirement = """
    Task: Analyze the roles of each Functional Module based on the Design Requirement, and reorganize the Initial Conceptual Solution.

    ## Design Requirement
    {requirement}

    ## Functional Modules
    {functional_modules}

    ## Initial Conceptual Solution
    {initial_conceptual_solution}

    The output should be formatted as a JSON instance that conforms to the JSON schema below.
    {CSD_format_instructions}
    """
    CSD_human_prompt = HumanMessagePromptTemplate.from_template(CSD_requirement)
    CSD_prompt = ChatPromptTemplate.from_messages([CSD_system_prompt, CSD_human_prompt])